UM MSBA - BGEN632

# Week 8: Advanced Data Manipulation

In the previous week, we covered approaches for filtering and querying data using various techniques. In this tutorial, we will go over how to perform advanced filtering and querying using `pandas` in Python. To start, let's set up our notebook.

### Notebook Setup

In [7]:
# import modules
import os
import numpy as np
import pandas as pd

In [46]:
# Set working directory to the folder (not the file!)
os.chdir("/Users/michellecalderwood/Documents/GitHub/week8labs/data")
os.getcwd()  # Confirm change

# Load the data file
ozone = pd.read_csv("ozone.data.txt")
ozone.head()

,rad\ttemp\twind\tozone
0,190\t67\t7.4\t41
1,118\t72\t8\t36
2,149\t74\t12.6\t12
3,313\t62\t11.5\t18
4,299\t65\t8.6\t23


In [48]:
# load data
ozone_df = pd.read_table("ozone.data.txt")
ozone_df.info()  # quick inspect

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rad     111 non-null    int64  
 1   temp    111 non-null    int64  
 2   wind    111 non-null    float64
 3   ozone   111 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 3.6 KB


In [50]:
(ozone_df
 .filter(["rad", "temp", "wind"])  # select desired columns
 .query("wind == 6.3")  # keep rows based on condition
 .head()  # display first n rows
)

,rad,temp,wind
39,267,92,6.3
47,285,84,6.3
69,51,79,6.3
80,237,96,6.3
81,188,94,6.3


You can see the similarities between the R code and Python code above. Note that we wrapped the entire Python code in parentheses `()` because otherwise we would need to place a backslash at the end of each line like so:

```Python
ozone_df \
 .filter(["rad", "temp", "wind"]) \
 .query("wind == 6.3") \
 .head() 
```

Another difference between pandas and tidyverse: lots of (single or double) quotes. Unlike tidyverse, which attempts to use as few quotes as possible, pandas relies on the underlying Python base which forces the use of quotes here. Could the pandas programmers have changed that? Sure. Yet, it does keep things Pythonic.

Let's use a different example that adds in more complexity. Here is an example with R code:

```R
ozone_data %>% 
    select(rad, temp, wind) %>%  # select desired columns
    filter(temp %in% 60:90) %>%  # keep rows based on value in specified range
    mutate(rad_wind = rad * wind) %>%  # create a new column based on mathematical operation applied to two other columns
    arrange(desc(rad_wind)) %>%  # sort the data largest to smallest based on value in new column
    head()  # display first n rows
```

Okay, now we'll convert the above example over to Python:

In [53]:
l = list(range(60, 91))  # create a range that we will use later for selecting rows

(ozone_df
 .filter(["rad", "temp", "wind"])  # select desired columns
 .query("temp in @l")  # keep rows based on value in specified range
 .assign(rad_wind = ozone_df.rad * ozone_df.wind)  # create a new column based on mathematical operation applied to two other columns
 .sort_values(by = ("rad_wind"), ascending = False) # sort the data largest to smallest based on value in new column
 .head(6)  # display first n rows
)

,rad,temp,wind,rad_wind
29,284,72,20.7,5878.8
17,320,73,16.6,5312.0
25,291,90,13.8,4015.8
93,259,76,15.5,4014.5
73,259,77,15.5,4014.5
11,334,64,11.5,3841.0


A couple of important points to consider. First, the range of numbers in `query()`. At first glance, it would seem appropriate to do this:

```Python
.query("temp in 62:89")
```
This would error out, unfortunately. The next idea might be performing the following:

```Python
.query("temp in list(range(60, 90))")
```

Python will return the error! 

As an alternative, we create the list of values outside the query and reference it inside the query. The variable `l` contains the list of numerical values: `l = list(range(60, 90))`.

Inside the `query()` function we can reference `l` by using the notation `@` like so: `.query("temp in @l")`. 

In fact, we can reference any variable outside the query using this notation. We can also use this notation to reference a variable in `filter()`.

The `in` operator evaluates the existence of some value inside a list. Unfortunately, we cannot create a list within the `query()` function using the function `range()`. As an alternative, we can explicitly state the list like so:

In [55]:
(ozone_df
 .filter(["rad", "temp", "wind"])
 .query("temp in [62, 64, 68, 90]")
 .assign(rad_wind = ozone_df.rad * ozone_df.wind)
 .sort_values(by = ("rad_wind"), ascending = False)
 .head(6)
)

,rad,temp,wind,rad_wind
25,291,90,13.8,4015.8
11,334,64,11.5,3841.0
14,322,68,11.5,3703.0
3,313,62,11.5,3599.5
102,238,64,12.6,2998.8
9,274,68,10.9,2986.6


We did not include all the possible values between 62 and 90, but you get the picture. 

One last important consideration here. With `dplyr`, when you use `group_by()` it is in conjunction with `summarize()`. The same applies here for pandas: when you use `groupby()` you need to pair it with `agg()`.


### Select Columns: `filter()`

Using tidyverse, the function `filter()` selects rows. In pandas, the function `filter()` selects columns.

Selecting columns is fairly straight forward. The syntax is:

```Python
.filter(["rad", "temp", "wind"])
```

*Be sure to encapsulate columns within square brackets and wrap column names with quotation marks*. In addition to listing column names, we can use regular expressions. Regex just has a way of creeping in when you least expect (or want) it. Think of it as a taste of things to come.

For example, if we only want columns that end with the letter *d* the syntax is:

```Python
(ozone_df
 .filter(regex = "d$")
)
```

### Select Rows: `query()`

The function `query()` is similar to tidyverse's `filter()` in that it specifies specific rows based on a set of criteria. We can use the following with `query()`:
* <
* \>
* <=
* \>=
* ==

We can also chain together multiple comparisons using the `&` operator for *and* and `|` operator for *or*. For example, instead of this expression:

```Python
.query("temp in @l")
```

We could write:

```Python
.query("temp >= 60 & temp <= 90")
```

The next set of operators are `in` and `not in`. In addition to the examples we have already covered, we can use these to compare columns against each other. For example, let's create a simple data frame:

In [57]:
df = pd.DataFrame({'a': np.random.randint(3, size = 12),
                   'b': np.random.randint(15, size = 12)})

df

,a,b
0,2,7
1,0,5
2,1,4
3,0,9
4,1,4
5,0,0
6,1,11
7,2,0
8,0,13
9,0,8


Now, let's query it to evaluate if a value in column `a` is contained in column `b`.

In [59]:
(df
 .query("a in b")
)

,a,b
0,2,7
1,0,5
3,0,9
5,0,0
7,2,0
8,0,13
9,0,8
10,0,13
11,2,2


The results show that only the values of 0 exist in column `b`.

### Arrange Rows: `sort_values()`

We can sort and order data based on columns in pandas using the function `sort_values()`. This allows us to sort a dataset given an ordered list of variables. The syntax is:

```Python
.sort_values(by = ["col_1", "col_2", "col_3", "etc"])
```

Note that this function sorts the data in ascending order by default. To sort the data in descending order, use `ascending = False` like so: 

```Python
.sort_values(by = ["col_1", "col_2", "col_3", "etc"], ascending = False)
```

Be sure to specify which direction to sort to get desired results!

If there are missing values in the data, we can place them at the top or end of the sort by using `na_position = "first"` for the top and `na_position = "last"` for the bottom.

### Add New Columns: `assign()`

We can create new columns of data from the existing columns in the DataFrame.

In [61]:
(df
 .assign(c = df.a * df.b + 23 - 3.67,
         d = lambda x: (x.b - x.c)*(3 + x.c))
)

,a,b,c,d
0,2,7,33.33,-956.5689
1,0,5,19.33,-319.9889
2,1,4,23.33,-508.9589
3,0,9,19.33,-230.6689
4,1,4,23.33,-508.9589
5,0,0,19.33,-431.6389
6,1,11,30.33,-644.2689
7,2,0,19.33,-431.6389
8,0,13,19.33,-141.3489
9,0,8,19.33,-252.9989


The notation is slightly different from the previous example. Note, we included the function `lambda x:` and reference each column using `x.b` or `x.c` instead of `df.b` or `df.c`. This is because the newly created column `c` does not actually exist in the data frame. The changes are saved in memory, but not the DataFrame `df`.

To preserve the simpler notation without using `lambda` we would have to save an intermediary data frame and then execute the second `assign()` function:

In [63]:
df_2 = (df
        .assign(c = df.a * df.b + 23 - 3.67)
       )

(df_2
 .assign(d = (df_2.b - df_2.c)*(3 + df_2.c))
)

,a,b,c,d
0,2,7,33.33,-956.5689
1,0,5,19.33,-319.9889
2,1,4,23.33,-508.9589
3,0,9,19.33,-230.6689
4,1,4,23.33,-508.9589
5,0,0,19.33,-431.6389
6,1,11,30.33,-644.2689
7,2,0,19.33,-431.6389
8,0,13,19.33,-141.3489
9,0,8,19.33,-252.9989


This defeats the purpose, though, of using the streamlined notation of pandas and piping. Yet, it is not wrong. It is very much still Pythonic.

### Rename Columns: `rename()`

To rename a column, we can use the function `rename()`. 

Each column changed is given in a pair with the original column name first, the new name second, separated by a colon `original_name:new_name`:

In [65]:
df_3 = (df_2
        .rename(columns={'a':'azure', 
                         'b':'blue',
                         'c':'cyan'})
       )

df_3

,azure,blue,cyan
0,2,7,33.33
1,0,5,19.33
2,1,4,23.33
3,0,9,19.33
4,1,4,23.33
5,0,0,19.33
6,1,11,30.33
7,2,0,19.33
8,0,13,19.33
9,0,8,19.33


As an alternative, if we do not want to create a new DataFrame, we can perform the operation in place like so:

In [67]:
(df_2
 .rename({'a':'azure',
          'b':'blue',
          'c':'cyan'},
         axis = 1,
         inplace = True)
)

df_2

,azure,blue,cyan
0,2,7,33.33
1,0,5,19.33
2,1,4,23.33
3,0,9,19.33
4,1,4,23.33
5,0,0,19.33
6,1,11,30.33
7,2,0,19.33
8,0,13,19.33
9,0,8,19.33


### Summaries of Data: `agg()`

The function `agg()` is an alias for *aggregate* `aggregate()`. The alias is preferred, though you can use both interchangeably. Sometimes we need to collapse many rows together by taking some kind of summary statistic. The `agg()` function allows for mutating data towards this goal. 

Below is a simple example of mutating each column using `sum` and `min`. Recall, `df_2` contains three columns of data, with columns `azure`, `blue`, and `cyan`.

We can compute the sum for each column as well as the minimum value:

In [69]:
(df_2
 .agg(["sum", "min"])
)

,azure,blue,cyan
sum,9,76,268.96
min,0,0,19.33


What if we are not interested in the minimum for column `b`? What if, instead, we want the maximum value? And, what if we do not want the maximum value for column `a`? In addition, we still want the sum for both columns. Oh, and one more request. What if we want the sum, maximum, and minimum for column `c`?

pandas has us covered. We can use `agg()` to do this, giving it the variables containing values to collapse and the functions used to aggregate:

In [71]:
(df_2
.agg({
    'azure':['sum', 'min'],
    'blue':['sum', 'max'],
    'cyan':['sum', 'min', 'max']
    })
)

,azure,blue,cyan
sum,9.0,76.0,268.96
min,0.0,NaN,19.33
max,NaN,13.0,33.33


---

That wraps up this week's tutorial! As always, review the content in this tutorial as needed. Proceed to the lab assignments once you feel ready to complete them.

Next week we will go over the implementation of statistical approaches in Python. See you then!